In [2]:
import os
os.chdir("/root/sli-product-recommendation")

import json
import random
import warnings
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder

from utils import *

warnings.filterwarnings('ignore')

seed = 42 
random.seed(seed)

In [3]:
yyyymm = '202305'
is_development = True if yyyymm=='202305' else False
target_label = 'target'
product_category = '상품중분류2'
print(f'Reference date: {yyyymm} \n  → is dev set: {is_development}')

Reference date: 202305 
  → is dev set: True


In [4]:
# save
# data_condition = ''
data_condition = 'delvariable_'
data_name = f'data_{data_condition}{yyyymm}'

In [5]:
def load_dataset(yyyymm):
    customer = pd.read_csv(f'data/CUST_BASE_{yyyymm}.csv', encoding='euc-kr', converters={'ID':str})
    if len(customer) != len(set(customer.ID)):
        raise ValueError('Duplicate values in ID')
    contract = pd.read_csv(f'data/PROD_BASE_{yyyymm}.csv', encoding='euc-kr', converters={'ID':str}).dropna()
    customer = customer.set_index('ID')
    contract['계약일자'] = pd.to_datetime(contract['계약일자'], format='%Y%m%d')
    return customer, contract

customer_raw, contract_raw = load_dataset(yyyymm)

def save_product_label(contract_previous, contract_target):
    contract_target_names = list(contract_target[product_category].value_counts().keys())
    contract_target_codes = list(range(1, len(contract_target_names)+1))
    contract_target_label = dict(zip(contract_target_names, contract_target_codes))
    
    contract_previous_names = pd.Series(contract_previous[product_category].value_counts().keys())
    added_contract_previous_names = contract_previous_names[~pd.Series(contract_previous_names).isin(contract_target_names)]
    start_added = max(contract_target_codes)
    added_contract_previous_codes = list(range(start_added+1, start_added+len(added_contract_previous_names)+1))
    added_contract_previous_label = dict(zip(added_contract_previous_names, added_contract_previous_codes))
    
    contract_previous_label = {**contract_target_label, **added_contract_previous_label}
    
    label_dictionary = {'contract_previous_label':contract_previous_label,
                        'contract_target_label':contract_target_label}
    
    with open('utils/target_label_dictionary.json', 'w') as f:
        json.dump(label_dictionary, f)

def contract_label_encoding(contract_previous, contract_target):
    if is_development:
        save_product_label(contract_previous, contract_target)
    label_dictionary = read_product_label()
    contract_previous_label = label_dictionary['contract_previous_label']
    contract_target_label = label_dictionary['contract_target_label']

    contract_previous['prdt_cat'] = contract_previous[product_category].replace(contract_previous_label)
    contract_target['prdt_cat'] = contract_target[product_category].replace(contract_target_label)
    return contract_previous, contract_target, contract_previous_label, contract_target_label

def contract_split(yyyymm, contract):  
    split_date = pd.to_datetime(yyyymm, format='%Y%m') + relativedelta(months=1)
    contract_previous = contract[pd.to_datetime(contract['계약일자'], format='%Y%m%d') < split_date]
    contract_target_raw = contract[pd.to_datetime(contract['계약일자'], format='%Y%m%d') >= split_date]
    
    contract_target = contract_target_raw[:]
    contract_target = contract_target[~contract_target.상품분류.isin(['34.실손', '80.미니보험'])]
    contract_target = contract_target[~contract_target.상품중분류2.isin(['34.실손', '36.단체', '기타'])]
    if 'smallcat' in data_name:
        contract_target = contract_target[~contract_target.상품중분류2.isin(['42.변액연금','12.변액종신','52.변액저축','20.CI', '35.정기', '33.상해'])]
    if 'delvariable' in data_name:
        contract_target = contract_target[~contract_target.상품중분류2.isin(['42.변액연금','12.변액종신','52.변액저축'])]
    if 'delmaxcat' in data_name:
        contract_target = contract_target[~contract_target.상품중분류2.isin(['38.건강'])]
    contract_target = contract_target.drop_duplicates()
    
    contract_previous, contract_target, contract_previous_label, contract_target_label = contract_label_encoding(contract_previous, contract_target)
    
    print('--- Contract Split ---')
    print(f'Number of total contracts: {contract.shape[0]:,}')
    print(f'Number of previous contracts: {contract_previous.shape[0]:,}')
    print(f'Number of target contract: {contract_target_raw.shape[0]:,} → {contract_target.shape[0]:,}')
    print(' Check the duration')
    print(f' - previous: {contract_previous.계약일자.min()} ~ {contract_previous.계약일자.max()}')
    print(f' - target: {contract_target.계약일자.min()} ~ {contract_target.계약일자.max()}')
    print()
    
    if max(contract_previous.계약일자) >= min(contract_target.계약일자):
        raise ValueError('Check')
    target_max_date = max(contract_target.계약일자)
    previous_max_date = max(contract_previous.계약일자)
    date_diff = (target_max_date - previous_max_date) / pd.Timedelta(days=1) / 30
    
    if date_diff == 6:
        raise ValueError('Check')
        
    return contract_previous, contract_target, contract_previous_label, contract_target_label   
    
contract_previous, contract_target, contract_previous_label, contract_target_label = contract_split(yyyymm, contract_raw)

def revise_ids(customer, contract_previous, contract_target):
    target_true_ids = set(contract_target.ID)
    previous_true_ids = set(contract_previous.ID)
    customer_ids = set(customer.index)
    common_ids = list(target_true_ids.intersection(customer_ids).intersection(previous_true_ids))
    
    customer_new = customer[customer.index.isin(common_ids)]
    contract_previous_new = contract_previous[contract_previous.ID.isin(common_ids)]
    contract_target_new = contract_target[contract_target.ID.isin(common_ids)]
    
    print('--- Find common IDs ---')
    print(f'Customer set: {len(customer):,} → {len(customer_new):,}')
    print(f'Contract Previous set: {len(contract_previous):,} → {len(contract_previous_new):,}')
    print(f'Contract Target set: {len(contract_target):,} → {len(contract_target_new):,}')
   
    return customer_new, contract_previous_new, contract_target_new

customer, contract_previous, contract_target = revise_ids(customer_raw, contract_previous, contract_target)

--- Contract Split ---
Number of total contracts: 332,970
Number of previous contracts: 268,327
Number of target contract: 64,643 → 54,984
 Check the duration
 - previous: 1979-03-31 00:00:00 ~ 2023-05-31 00:00:00
 - target: 2023-06-01 00:00:00 ~ 2023-10-31 00:00:00

--- Find common IDs ---
Customer set: 45,417 → 39,928
Contract Previous set: 268,327 → 251,937
Contract Target set: 54,984 → 53,899


### 고객 메타정보

In [6]:
# for field 
f_num = ['계약자연령','추정소득','최근계약경과월']
f_cat = ['계약자성별', '외국인여부', 'BP상태코드', '컨설턴트여부', '임직원여부', '관심고객여부', 'VIP등급', '우량직종여부', '직업대분류', '직업군_관계사공통기준', 
         '투자성향', '업종1', '업종2'] # , '업종3', '업종4', '업종5', '업종6'
f_fire_contract = ['F00003','F00004','F00005','F00006','F00007','F00008','F00009','F00010','F00011','F00012']

f_stock = list(filter(lambda x: x[0]=='S', customer.columns))
f_fire = list(filter(lambda x: x[0]=='F', customer.columns))
f_card = list(filter(lambda x: x[0]=='C', customer.columns))

customer['CD0001'] = customer['CD0001'].fillna('-1')
customer[f_stock+f_fire+f_card] = customer[f_stock+f_fire+f_card].replace(-99999999, 0).fillna(0)

In [44]:
print(dtype_numeric)

['계약자연령', '추정소득', '최근계약경과월', 'S00006', 'S00007', 'S00008', 'S00009', 'S00010', 'S00011', 'S00012', 'S00013', 'S00014', 'S00015', 'S00016', 'S00017', 'S00018', 'S00019', 'S00020', 'S00021', 'S00022', 'S00023', 'S00024', 'S00025', 'S00026', 'S00027', 'S00028', 'S00029', 'S00030', 'S00031', 'S00032', 'S00033', 'S00034', 'S00035', 'S00036', 'S00037', 'S00038', 'S00039', 'S00040', 'S00041', 'S00042', 'S00043', 'S00044', 'S00045', 'S00046', 'S00047', 'S00048', 'S00049', 'S00050', 'S00051', 'S00052', 'S00053', 'S00054', 'S00055', 'F00003', 'F00004', 'F00005', 'F00006', 'F00007', 'F00008', 'F00009', 'F00010', 'F00011', 'F00012', 'F00013', 'F00014', 'F00015', 'F00016', 'F00017', 'F00018', 'F00019', 'F00020', 'F00021', 'F00022', 'F00023', 'F00024', 'F00025', 'F00026', 'F00027', 'F00028', 'F00029', 'F00030', 'F00031', 'F00032', 'F00033', 'F00034', 'F00035', 'F00036', 'F00037', 'F00038', 'F00039', 'F00040', 'F00041', 'F00042', 'F00043', 'F00044', 'F00045', 'F00046', 'F00047', 'F00048', 'F00049', '

In [7]:
# encoding
dtype_object = f_cat + customer[f_stock+f_fire+f_card].select_dtypes('object').columns.to_list()
dtype_numeric = f_num + customer[f_stock+f_fire+f_card].select_dtypes(exclude=['object']).columns.to_list()
assert len(dtype_object) + len(dtype_numeric) == customer.shape[1]-4-4 # '마감년월','증권정보등록여부','소비성향대상','NO_CARD_CUST'
customer[dtype_object] = customer[dtype_object].fillna('-1')

data_cust = customer[dtype_object + dtype_numeric]

if is_development:
    os.makedirs('utils/encoders/', exist_ok=True)
    # category
    for feat in dtype_object:
        lbe = LabelEncoder()
        data_cust[feat] = lbe.fit_transform(data_cust[feat])
        # save
        filename = f'utils/encoders/{feat}_label_encoder.joblib'
        joblib.dump(lbe, filename)
        
    # numeric
    mms = MinMaxScaler(feature_range=(0, 1))
    data_cust[dtype_numeric] = mms.fit_transform(data_cust[dtype_numeric])
    joblib.dump(mms, 'utils/encoders/minmax_scaler.joblib')
    
else:
    for feat in dtype_object:
        filename = f'utils/encoders/{feat}_label_encoder.joblib'
        lbe = joblib.load(filename)
        data_cust[feat] = data_cust[feat].map(
            lambda s: lbe.transform([s])[0]
            if s in lbe.classes_
            else -1)
        
    mms = joblib.load('utils/encoders/minmax_scaler.joblib')
    data_cust[dtype_numeric] = mms.transform(data_cust[dtype_numeric])

data_cust.shape

(39928, 281)

In [8]:
ids  = data_cust.drop(data_cust.columns, axis=1)
len(ids)

39928

# 기계약정보

In [9]:
# 최근 N개 계약
def latest_n_product(contract_previous, n=10):
    latest_product_order = contract_previous.sort_values(by=['ID', '계약일자'], ascending=[True, False])
    latest_product_order = latest_product_order.groupby(['ID']).head(n)
    latest_product_order['order'] = latest_product_order.groupby(['ID']).cumcount() + 1    
    
    latest_product_category = latest_product_order.pivot_table(index='ID', columns='order', values='prdt_cat')
    latest_product_category.columns = list(map(lambda x: f'product_{x+1}', range(n)))
    return latest_product_category

latest_product_category = latest_n_product(contract_previous).fillna(0)
latest_product_category.head(1)

,product_1,product_2,product_3,product_4,product_5,product_6,product_7,product_8,product_9,product_10
ID,,,,,,,,,,
1,6.0,11.0,9.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
end_date = pd.to_datetime(yyyymm, format='%Y%m').to_pydatetime().date()

contract_previous['계약여부'] = 1
contract_previous['계약경과월'] = list(map(lambda x: relativedelta(end_date, x).years*12 + relativedelta(end_date, x).months, contract_previous['계약일자']))
contract_previous.head(1)

,ID,상품코드,상품분류,상품중분류2,판매상품코드,계약일자,prdt_cat,계약여부,계약경과월
0,1,LP0673001,32.암,32.암,P0673,2022-10-18,6,1,6


In [11]:
contract_previous_features = ids[:]
contract_previous_features['pre_contract_count'] = contract_previous.groupby('ID').계약여부.count()
contract_previous_features['pre_contract_month_min'] = contract_previous.groupby('ID').계약경과월.min()
contract_previous_features['pre_contract_month_max'] = contract_previous.groupby('ID').계약경과월.max()

assert len(contract_previous_features[contract_previous_features.pre_contract_count==0]) == 0

In [12]:
# YN by product categoies
data_prodpre_yn = contract_previous.pivot_table(index='ID', columns='prdt_cat', values='계약여부', aggfunc='max').fillna(0)
data_prodpre_yn.columns = list(map(lambda x: f'pre_yn_{x}', data_prodpre_yn.columns))
# data_prodpre_yn.head(1)

In [13]:
# Minimum elapsed months by product categories
data_prodpre_month = contract_previous.pivot_table(index='ID', columns='prdt_cat', values='계약경과월', aggfunc='min')#.fillna(-1)
data_prodpre_month.columns = list(map(lambda x: f'pre_m_{x}', data_prodpre_month.columns))

In [14]:
# make quantile dictionary
def make_elapsed_month_quantile_dictionary(data_prodpre_month, yyyymm, inital_quantile_num=10):
    if yyyymm != '202302':
        raise ValueError('Check data')
    quantile_dic = {}
    for col in data_prodpre_month.columns:
        n = inital_quantile_num
        quantiles = data_prodpre_month[col].quantile(list(np.linspace(0, 1, n+1)))
        while quantiles.duplicated().any():
            n = n-1
            quantiles = data_prodpre_month[col].quantile(list(np.linspace(0, 1, n+1)))
        quantile_dic[col] = quantiles.to_dict()
    with open(f'utils/elapsed_month_quantile_dictionary_{yyyymm}.json', 'w') as f:
        json.dump(quantile_dic, f)
# make_elapsed_month_quantile_dictionary(data_prodpre_month, yyyymm)

with open('utils/elapsed_month_quantile_dictionary_202302.json', 'rb') as f:
    quantile_dic = json.load(f)

In [15]:
# months → quantile categories
data_prodpre_month_cat = pd.DataFrame()
for col, bins in quantile_dic.items():
    data_prodpre_month_cat[col+'_cat'] = pd.cut(data_prodpre_month[col], bins=list(bins.values()), labels=list(map(lambda x: f'Q{x+1}', range(len(bins)-1))))
assert data_prodpre_month_cat.shape == data_prodpre_month.shape

In [16]:
data_prodpre_month = data_prodpre_month.fillna(-1)
for feat in data_prodpre_month_cat:
    lbe = LabelEncoder()
    data_prodpre_month_cat[feat] = lbe.fit_transform(data_prodpre_month_cat[feat])

In [17]:
data_prodpre = pd.concat([ids, latest_product_category, contract_previous_features, data_prodpre_yn, data_prodpre_month, data_prodpre_month_cat], axis=1)
assert data_prodpre.shape[0] == customer.shape[0]

In [18]:
a = data_prodpre.isna().sum(axis=0)
assert len(a[a>0]) == 0

# 타겟 계약 정보

In [19]:
target_category_code = list(contract_target_label.values())

In [20]:
def make_target_describe(contract_target_y):
    # count
    traget_cnt_dict = contract_target_y[product_category].value_counts().to_dict()

    # percent
    total_count = sum(traget_cnt_dict.values())
    target_precent = {}
    for k, v in traget_cnt_dict.items():
        target_precent[k] = round(v / total_count*100, 2)
    target_precent = {key:value for key, value in sorted(target_precent.items(), key=lambda x:x[1], reverse=True)}
    
    return traget_cnt_dict, target_precent

In [21]:
contract_target_y = contract_target[['ID', product_category,'prdt_cat']].drop_duplicates()
if 'over' not in data_name:    
    traget_cnt_dict, target_precent = make_target_describe(contract_target_y)
    contract_target_y = contract_target_y.drop(columns=product_category)

In [22]:
if 'over' in data_name:
    value_count = contract_target_y.prdt_cat.value_counts()
    max_count = value_count.max()
    contract_target_y_over = contract_target_y[:]
    for code in target_category_code:
        limit_n = max_count//5
        value_n = value_count[code]
        if value_n < limit_n and value_n>1000:
            df = contract_target_y_over[contract_target_y_over.prdt_cat == code]
            n = min(limit_n-df.shape[0], df.shape[0]*3)
            random_rows = df.sample(n=n, replace=True)
            contract_target_y_over = pd.concat([contract_target_y_over, random_rows], ignore_index=True)   
               
    traget_cnt_dict, target_precent = make_target_describe(contract_target_y_over)
    contract_target_y = contract_target_y_over.drop(columns=product_category)

### label

In [34]:
read_product_label()

{'contract_previous_label': {'38.건강': 1,
  '11.일반종신': 2,
  '31.어린이': 3,
  '37.치아': 4,
  '41.일반연금': 5,
  '32.암': 6,
  '43.적격연금': 7,
  '51.일반저축': 8,
  '39.간병': 9,
  '20.CI': 10,
  '35.정기': 11,
  '33.상해': 12,
  '12.변액종신': 13,
  '34.실손': 14,
  '기타': 15,
  '42.변액연금': 16,
  '52.변액저축': 17,
  '36.단체': 18},
 'contract_target_label': {'38.건강': 1,
  '11.일반종신': 2,
  '31.어린이': 3,
  '37.치아': 4,
  '41.일반연금': 5,
  '32.암': 6,
  '43.적격연금': 7,
  '51.일반저축': 8,
  '39.간병': 9,
  '20.CI': 10,
  '35.정기': 11,
  '33.상해': 12}}

In [23]:
if is_development:
    tot_ids = set(contract_target_y.ID)
    contract_target_n = pd.DataFrame()
    print('Code: target / non-target')
    for code in target_category_code:
        code_target_data = contract_target_y[contract_target_y['prdt_cat'] == code].drop_duplicates()
        code_target_ids = set(code_target_data.ID)
        if 'over' not in data_name:
            assert len(code_target_ids) == len(set(code_target_ids))

        categroy_non_target_data = contract_target_y.loc[~contract_target_y['ID'].isin(code_target_ids), ['ID']].drop_duplicates()

        num_of_target = len(code_target_ids)
        num_of_non_target = len(tot_ids-code_target_ids)
        
        print(f'{code}: {num_of_target:,} / {num_of_non_target:,}')
        num_of_sample = min(num_of_non_target, num_of_target*2)
        categroy_non_target_data = categroy_non_target_data.sample(num_of_sample, replace=False, random_state=seed)
        categroy_non_target_data['prdt_cat'] = code

        contract_target_n = contract_target_n.append(categroy_non_target_data)

        del categroy_non_target_data
    
    print(contract_target_n.shape)
    assert len(contract_target_n[contract_target_n.duplicated()]) == 0    

Code: target / non-target
1: 25,987 / 13,941
2: 11,900 / 28,028
3: 3,352 / 36,576
4: 2,135 / 37,793
5: 1,329 / 38,599
6: 992 / 38,936
7: 789 / 39,139
8: 534 / 39,394
9: 327 / 39,601
10: 244 / 39,684
11: 195 / 39,733
12: 66 / 39,862
(57667, 2)


In [24]:
contract_target_y[target_label] = 1
if is_development:
    contract_target_n[target_label] = 0
    data_target_label = contract_target_y.append(contract_target_n)
else:
    data_target_label = contract_target_y[:]
    
data_target_label = data_target_label.set_index('ID')
data_target_label.columns = ['target_category', target_label]

In [25]:
# has item
has_item = list(map(lambda x, y: data_prodpre.loc[str(x), f'pre_yn_{y}'], data_target_label.index, data_target_label.target_category))
assert len(has_item) == data_target_label.shape[0]
data_target_label['has_item'] = has_item

In [26]:
a = data_target_label.isna().sum(axis=0)
assert len(a[a>0]) == 0

# Join

In [27]:
data = data_cust.join(data_prodpre).join(data_target_label)
data.shape

(105517, 351)

In [28]:
a = data.isna().sum(axis=0)
assert len(a[a>0]) == 0
assert data.duplicated().sum() == 0

# SAVE

In [29]:
num_of_rows = data.shape[0]
num_of_ids = len(set(data.index))

target_ = data[data[target_label]==1]
num_of_target = sum(target_[target_label])
num_of_target_ids = len(set(target_.index))

data_config = {}
data_config['data_name'] = data_name

data_config['data'] = {'reference_date':yyyymm,
                       'is_development':is_development,
                        'num_of_rows':num_of_rows,
                        'num_of_ids':num_of_ids}

data_config['target'] = {'target_label':target_label,
                        'num_of_target':num_of_target,
                        'percent_of_target': round(num_of_target/num_of_rows*100,1),
                        'num_of_target_ids':num_of_target_ids,
                        'percent_of_target_ids':round(num_of_target_ids/num_of_ids*100,1),
                        'target_categories':list(contract_target.상품중분류2.value_counts().keys()),
                        'target_category_count':traget_cnt_dict,
                        'target_category_percent':target_precent
                         }
data_config['features'] = {'user_numeric':f_num,
                           # 'features_card':f_card,
                           # 'features_fire':f_fire,
                           # 'features_stock':f_stock,
                           'f_fire_contract':f_fire_contract,
                           'product_describe':contract_previous_features.columns.tolist(),
                           'product_yn':data_prodpre_yn.columns.tolist(),
                           'product_month':data_prodpre_month.columns.tolist(),
                           'product_month_category':data_prodpre_month_cat.columns.tolist(),
                           'user_category':f_cat,
                           'product_latest':latest_product_category.columns.tolist(),
                           'has_item':['has_item'],
                           'target_category':['target_category'],
                           }


with open(f'input/{data_name}_config.json', 'w') as f:
    json.dump(data_config, f)

In [30]:
data.to_csv(f'input/{data_name}.csv')

In [31]:
data.target.value_counts()

0    57667
1    47850
Name: target, dtype: int64

In [32]:
# load
devset = pd.read_csv('input/data_delvariable_202305.csv')


In [39]:
grp = devset.groupby('target_category').agg({'target':['sum', 'count']})
grp.columns = ['target_sum', 'target_count']
grp['proportion'] = round(grp.target_sum / grp.target_count*100,1)
grp

,target_sum,target_count,proportion
target_category,,,
1,25987,39928,65.1
2,11900,35700,33.3
3,3352,10056,33.3
4,2135,6405,33.3
5,1329,3987,33.3
6,992,2976,33.3
7,789,2367,33.3
8,534,1602,33.3
9,327,981,33.3
